## Install dependencies

In [1]:
# we have to enter OpenAI key here and then we can run all cells freely...
import os
from getpass import getpass
from openai import OpenAI

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key:  ········


In [42]:
# Homework: Agents
# In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

# Preparation
# First, we'll define a function that we will use when building our agent.

# It will generate fake weather data:

import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:    
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

# def get_weather(*args, **kwargs):
#     print("DEBUG:", *args, *[f"{k}={v}" for k, v in kwargs.items()])


## Q1. Define function description

In [43]:
# Q1. Define function description
# We want to use it as a tool for our agent, so we need to describe it

# How should the description for this function look like? Fill in missing parts

get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "get weather data",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "get weather data from known city and generate for unknown"
            }
        },
        #"required": ["query"],
        "additionalProperties": False
    }
}

# What did you put in TODO3? "city"


In [4]:
# Testing it (Optional)
# If you have OpenAI API Key (or alternative provider), let's test it.

# A question could be "What's the weather like in Germany?"

# Experiment with different system prompts to have better answers from the system.

# You can use chat_assistant.py or implement everything yourself

#!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

#!pip install markdown --q

In [24]:
from IPython.display import display, HTML
import markdown # pip install markdown

In [25]:
# connecting to LLM, we entered our API KEY already at the first cell
client = OpenAI()

In [44]:
# Let's use it:

import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

developer_prompt = """
You're a weather bureau assistant. 
You're given a question about current weather conditions in particular city or country and your task is to answer it.

Use weather tool to answer the question. Parse the user question and pass only the name of the city to the tool.
If you unsure what is the city user asked for, double-check exact city name.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)


In [45]:
# and run
# type stop to exit

chat.run()

You: What is the weather in Prague?


You: exit


You: stop


Chat ended.
